## 安装 condoa
```bash
 conda create -n ai-learn python=3.12
 conda activate ai-learn
```

In [ ]:
# 执行以下命令安装必要的包
%pip install -U langchain=="0.3.23"

In [ ]:
# 验证langchain版本
import langchain 
print(langchain.__version__)
assert langchain.__version__ == "0.3.23"

0.3.23
